In [3]:
"""Importing relevant libraries and data"""
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
# 1 -- Import dask as well as matplotlib for generating graphics
from matplotlib import pyplot as plt
# 2 -- Reads the 2017 CSV file into a Dask Dataframe
df = dd.read_csv('./data/nyu_parking_data/parking_violations_2017.csv')
df

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,Street Code3,Vehicle Expiration Date,Violation Location,Violation Precinct,Issuer Precinct,Issuer Code,Issuer Command,Issuer Squad,Violation Time,Time First Observed,Violation County,Violation In Front Of Or Opposite,House Number,Street Name,Intersecting Street,Date First Observed,Law Section,Sub Division,Violation Legal Code,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
npartitions=33,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,object,object,object,object,int64,object,object,object,int64,int64,int64,int64,float64,int64,int64,int64,object,object,object,object,object,object,object,object,object,int64,int64,object,object,object,object,object,object,float64,int64,object,int64,object,object,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [4]:
class Cleaning():
    def __init__(self, dataframe):
        self.dataframe = dataframe
        
    def calc_miss_val_percent(self):
        """
        - Counting missing values in the DataFrame
        - Calculating the percent of missing values in the DataFrame
        - Computing the DAG
        """

        missing_values = self.dataframe.isnull().sum()
        missing_count = ((missing_values / self.dataframe.index.size) * 100)

        with ProgressBar():
            missing_count_pct = missing_count.compute()

        return missing_count_pct        

    
    def filter_sparse_columns(self, missing_count_pct, treshold):
        """Filtering sparse columns"""

        columns_to_drop = missing_count_pct[missing_count_pct > treshold].index
        with ProgressBar():
            df_dropped = self.dataframe.drop(columns_to_drop, axis=1).persist()

        return df_dropped

In [7]:
cleaner = Cleaning(df)

miss_count_pct = cleaner.calc_miss_val_percent()

[####################################    ] | 92% Completed | 39.5s

/home/echoappletree/anaconda3/envs/daskenv/lib/python3.9/multiprocessing/pool.py:125: DtypeWarning: Columns (18,38) have mixed types.Specify dtype option on import or set low_memory=False.
  result = (True, func(*args, **kwds))


[########################################] | 100% Completed | 40.3s


In [8]:
miss_count_pct

Summons Number                         0.000000
Plate ID                               0.006739
Registration State                     0.000000
Plate Type                             0.000000
Issue Date                             0.000000
Violation Code                         0.000000
Vehicle Body Type                      0.395361
Vehicle Make                           0.676199
Issuing Agency                         0.000000
Street Code1                           0.000000
Street Code2                           0.000000
Street Code3                           0.000000
Vehicle Expiration Date                0.000000
Violation Location                    19.183510
Violation Precinct                     0.000000
Issuer Precinct                        0.000000
Issuer Code                            0.000000
Issuer Command                        19.093212
Issuer Squad                          19.101506
Violation Time                         0.000583
Time First Observed                   92

In [9]:
df_dropped = cleaner.filter_sparse_columns(miss_count_pct, 60)

[#####################################   ] | 93% Completed | 31.5s

/home/echoappletree/anaconda3/envs/daskenv/lib/python3.9/site-packages/dask/core.py:121: DtypeWarning: Columns (18,38) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*(_execute_task(a, cache) for a in args))


[########################################] | 100% Completed | 32.1s
